In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [11]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...


True

In [19]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('chatbot-intents.json').read())

words=[]
classes = []
documents = []
ignore_letters = ['?', '!', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('chatbot-words.pkl', 'wb'))
pickle.dump(classes, open('chatbot-classes.pkl', 'wb'))

In [16]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

<ipython-input-16-6bee798cfc15>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [17]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot-model.h5', hist)

print("model created")

C:\Users\Lenovo\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
5/5 [==============================] - 2s 17ms/step - loss: 1.4530 - accuracy: 0.2273
Epoch 2/200
5/5 [==============================] - 0s 2ms/step - loss: 1.4141 - accuracy: 0.3636
Epoch 3/200
5/5 [==============================] - 0s 13ms/step - loss: 1.3413 - accuracy: 0.3636
Epoch 4/200
5/5 [==============================] - 0s 4ms/step - loss: 1.3046 - accuracy: 0.4545
Epoch 5/200
5/5 [==============================] - 0s 2ms/step - loss: 1.3058 - accuracy: 0.5000
Epoch 6/200
5/5 [==============================] - 0s 3ms/step - loss: 1.2639 - accuracy: 0.5909
Epoch 7/200
5/5 [==============================] - 0s 3ms/step - loss: 1.2269 - accuracy: 0.5000
Epoch 8/200
5/5 [==============================] - 0s 3ms/step - loss: 1.1935 - accuracy: 0.6818
Epoch 9/200
5/5 [==============================] - 0s 3ms/step - loss: 1.0819 - accuracy: 0.6818
Epoch 10/200
5/5 [==============================] - 0s 3ms/step - loss: 1.0945 - accuracy: 0.6364
Epoch 11/200
5/5 [=========